<a href="https://colab.research.google.com/github/jyotidabass/Chatbot/blob/main/Simple_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install sqlite3 nltk
import sqlite3

conn = sqlite3.connect("candidates.db")
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS candidates (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                skills TEXT,
                budget INTEGER,
                full_time INTEGER,
                github_username TEXT,
                experience_at_big_tech INTEGER
            )''')

c.execute("INSERT INTO candidates (name, skills, budget, full_time, github_username, experience_at_big_tech) VALUES (?, ?, ?, ?, ?, ?)",
          ("John Doe", "Python, JavaScript", 50000, 1, "johndoe", 1))
c.execute("INSERT INTO candidates (name, skills, budget, full_time, github_username, experience_at_big_tech) VALUES (?, ?, ?, ?, ?, ?)",
          ("Jane Doe", "React, Node.js", 30000, 0, "janedoe", 0))
c.execute("INSERT INTO candidates (name, skills, budget, full_time, github_username, experience_at_big_tech) VALUES (?, ?, ?, ?, ?, ?)",
          ("Bob Smith", "AWS, Java", 70000, 1, "bobsmith", 1))

conn.commit()
conn.close()

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import sqlite3
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import re

def process_query(query):
    # Tokenize the query
    tokens = word_tokenize(query)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Create a frequency distribution of the tokens
    fdist = FreqDist(lemmatized_tokens)

    # Map the tokens to their corresponding categories (skills, budget, etc.)
    categories = defaultdict(int)
    for token, freq in fdist.items():
        if re.match(r'\d+', token):
            categories['budget'] += freq
        elif token in ('full-time', 'part-time'):
            categories['full_time'] += freq
        elif token in ('python', 'react', 'aws'):
            categories['skills'] += freq
        elif token in ('big tech', 'google', 'facebook', 'amazon', 'microsoft'):
            categories['experience_at_big_tech'] += freq
        # Removed the 'else' block to avoid the 'other' category
        # else:
        #     categories['other'] += freq

    # Return the processed categories
    return categories

def get_candidates(skills=None, budget=None, full_time=None, github_username=None, experience_at_big_tech=None):
    # Set default values for all parameters to handle cases when they are not provided
    query_str = """
                SELECT * FROM candidates
                WHERE 1=1
                """
    params = ()

    if skills:
        query_str += " AND skills LIKE ?" # Use a placeholder for the LIKE parameter
        params += (f'%{skills}%',) # Pass the formatted skill string as a parameter
    if budget:
        query_str += " AND budget >= ?"
        params += (budget,)
    if full_time:
        query_str += " AND full_time = ?"
        params += (full_time,)
    if github_username:
        query_str += " AND github_username = ?"
        params += (github_username,)
    if experience_at_big_tech:
        query_str += " AND experience_at_big_tech = ?"
        params += (experience_at_big_tech,)

    return query_str, params

if __name__ == '__main__':
    # Connect to the SQLite database
    conn = sqlite3.connect('candidates.db')
    c = conn.cursor()

    # Prompt the user for the query
    while True:
        query = input("What are you looking for? ")
        categories = process_query(query)

        if not categories:
            print("Sorry, I need more information to assist you.")
        else:
            query_str, params = get_candidates(**categories) # This should work now as 'other' is not in categories and missing arguments are handled
            c.execute(query_str, params)
            candidates = c.fetchall()
            if candidates:
                print("Here are some candidates:")
                for candidate in candidates:
                    print(f"Name: {candidate[1]}, Skills: {candidate[2]}, Budget: {candidate[3]}, Full-Time: {candidate[4]}, GitHub Username: {candidate[5]}, Experience at Big Tech: {candidate[6]}")
            else:
                print("No candidates found.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


What are you looking for? candidate
Sorry, I need more information to assist you.
What are you looking for? Skills
Sorry, I need more information to assist you.
What are you looking for? Budget
Sorry, I need more information to assist you.
What are you looking for? Full-Time
Here are some candidates:
Name: John Doe, Skills: Python, JavaScript, Budget: 50000, Full-Time: 1, GitHub Username: johndoe, Experience at Big Tech: 1
Name: Bob Smith, Skills: AWS, Java, Budget: 70000, Full-Time: 1, GitHub Username: bobsmith, Experience at Big Tech: 1
Name: John Doe, Skills: Python, JavaScript, Budget: 50000, Full-Time: 1, GitHub Username: johndoe, Experience at Big Tech: 1
Name: Bob Smith, Skills: AWS, Java, Budget: 70000, Full-Time: 1, GitHub Username: bobsmith, Experience at Big Tech: 1
What are you looking for? thanks
Sorry, I need more information to assist you.
What are you looking for? Java
Sorry, I need more information to assist you.
What are you looking for? Python
No candidates found.
